# Nok's experiment with nbdev and DebugRunner

> Demo

In [ ]:
#| default_exp core

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# | export
from collections import Counter
from itertools import chain
from typing import Any, Dict, Iterable, List, Set

from kedro.framework.hooks.manager import _NullPluginManager
from kedro.io import AbstractDataSet, DataCatalog, MemoryDataSet
from kedro.pipeline import Pipeline
from kedro.pipeline.node import Node
from kedro.runner import SequentialRunner
from kedro.runner.runner import AbstractRunner, run_node
from pluggy import PluginManager


class DebugRunner(SequentialRunner):
    def run(
        self,
        pipeline: Pipeline,
        catalog: DataCatalog,
        dataset_names: List[str] = None,
        hook_manager: PluginManager = None,
        session_id: str = None,
    ) -> Dict[str, Any]:
        """Run the ``Pipeline`` using the datasets provided by ``catalog``
        and save results back to the same objects.

        Args:
            pipeline: The ``Pipeline`` to run.
            catalog: The ``DataCatalog`` from which to fetch data.
            hook_manager: The ``PluginManager`` to activate hooks.
            session_id: The id of the session.

        Raises:
            ValueError: Raised when ``Pipeline`` inputs cannot be satisfied.

        Returns:
            Any node outputs that cannot be processed by the ``DataCatalog``.
            These are returned in a dictionary, where the keys are defined
            by the node outputs.

        """
        if not dataset_names:
            dataset_names = []
        hook_manager = hook_manager or _NullPluginManager()
        catalog = catalog.shallow_copy()

        unsatisfied = pipeline.inputs() - set(catalog.list())
        if unsatisfied:
            raise ValueError(
                f"Pipeline input(s) {unsatisfied} not found in the DataCatalog"
            )

        free_outputs = (
            pipeline.outputs()
        )  # Return everything regardless if it it's in catalog
        unregistered_ds = pipeline.data_sets() - set(catalog.list())
        for ds_name in unregistered_ds:
            catalog.add(ds_name, self.create_default_data_set(ds_name))

        if self._is_async:
            self._logger.info(
                "Asynchronous mode is enabled for loading and saving data"
            )
        self._run(pipeline, catalog, dataset_names, hook_manager, session_id)

        self._logger.info("Pipeline execution completed successfully.")
        
        free_outputs = free_outputs | set(dataset_names)  # Union

        return {ds_name: catalog.load(ds_name) for ds_name in free_outputs}

    def _run(
        self,
        pipeline: Pipeline,
        catalog: DataCatalog,
        dataset_names: List[str],
        hook_manager: PluginManager,
        session_id: str = None,
    ) -> None:
        """The method implementing sequential pipeline running.

        Args:
            pipeline: The ``Pipeline`` to run.
            catalog: The ``DataCatalog`` from which to fetch data.
            hook_manager: The ``PluginManager`` to activate hooks.
            session_id: The id of the session.

        Raises:
            Exception: in case of any downstream node failure.
        """
        nodes = pipeline.nodes
        done_nodes = set()

        load_counts = Counter(chain.from_iterable(n.inputs for n in nodes))

        for exec_index, node in enumerate(nodes):
            try:
                run_node(node, catalog, hook_manager, self._is_async, session_id)
                done_nodes.add(node)
            except Exception:
                self._suggest_resume_scenario(pipeline, done_nodes, catalog)
                raise

            # decrement load counts and release any data sets we've finished with
            for data_set in node.inputs:
                load_counts[data_set] -= 1
                if load_counts[data_set] < 1 and data_set not in pipeline.inputs():
                    if data_set not in dataset_names:
                        catalog.release(data_set)
            for data_set in node.outputs:
                if load_counts[data_set] < 1 and data_set not in pipeline.outputs():
                    if data_set not in dataset_names:
                        catalog.release(data_set)

            self._logger.info(
                "Completed %d out of %d tasks", exec_index + 1, len(nodes)
            )


In [ ]:
# `DebugRunner` has to be used in a different way since `session.run` don't support additional argument, so we are going to use a lower level approach and construct `Runner` and `Pipeline` and `DataCatalog` ourselves.

# Testing Kedro Project: https://github.com/noklam/kedro_gallery/tree/master/kedro-debug-runner-demo
%load_ext kedro.ipython
%reload_kedro ~/dev/kedro_gallery/kedro-debug-runner-demo

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[10/06/22 15:32:15] INFO     Updated path to Kedro project:                                          ]8;id=390030;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=425671;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py#54\54]8;;\
                             /Users/Nok_Lam_Chan/dev/kedro_gallery/kedro-debug-runner-demo                         

[10/06/22 15:32:17] INFO     Kedro project kedro_debug_runner_demo                                   ]8;id=810861;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=940736;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py#77\77]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines' ]8;id=577165;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=324475;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py#78\78]8;;\

In [ ]:
%reload_kedro ~/dev/kedro_gallery/kedro-debug-runner-demo
runner = DebugRunner()
default_pipeline = pipelines["__default__"]
run_1 = runner.run(default_pipeline, catalog)


                    INFO     Updated path to Kedro project:                                          ]8;id=683543;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=725273;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py#54\54]8;;\
                             /Users/Nok_Lam_Chan/dev/kedro_gallery/kedro-debug-runner-demo                         

[10/06/22 15:32:19] INFO     Kedro project kedro_debug_runner_demo                                   ]8;id=926857;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=802456;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py#77\77]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines' ]8;id=580655;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=442814;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/ipython/__init__.py#78\78]8;;\

                    INFO     Loading data from 'example_iris_data' (CSVDataSet)...              ]8;id=125006;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=754959;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'parameters' (MemoryDataSet)...                  ]8;id=393350;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=372618;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

...

                    INFO     Saving data to 'y_pred' (MemoryDataSet)...                         ]8;id=127495;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=139301;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#382\382]8;;\

                    INFO     Loading data from 'y_pred' (MemoryDataSet)...                      ]8;id=391340;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=271877;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'y_test' (MemoryDataSet)...                      ]8;id=206032;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=523569;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Running node: report_accuracy: report_accuracy([y_pred,y_test]) -> None    ]8;id=801510;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py\node.py]8;;\:]8;id=309475;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py#327\327]8;;\

                    INFO     Model has accuracy of 0.933 on test data.                                  ]8;id=34142;file:///Users/Nok_Lam_Chan/dev/kedro_gallery/kedro-debug-runner-demo/src/kedro_debug_runner_demo/nodes.py\nodes.py]8;;\:]8;id=465017;file:///Users/Nok_Lam_Chan/dev/kedro_gallery/kedro-debug-runner-demo/src/kedro_debug_runner_demo/nodes.py#74\74]8;;\

In [ ]:
runner = DebugRunner()
default_pipeline = pipelines["__default__"]
run_2 = runner.run(default_pipeline, catalog, dataset_names=["example_iris_data"])


                    INFO     Loading data from 'example_iris_data' (CSVDataSet)...              ]8;id=898218;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=462269;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'parameters' (MemoryDataSet)...                  ]8;id=970439;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=874751;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Running node: split: split_data([example_iris_data,parameters]) ->         ]8;id=917951;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py\node.py]8;;\:]8;id=154413;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py#327\327]8;;\
                             [X_train,X_test,y_train,y_test]                                                       

                    INFO     Saving data to 'X_train' (MemoryDataSet)...                        ]8;id=325731;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=281974;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#382\382]8;;\

                    INFO     Saving data to 'X_test' (MemoryDataSet)...                         ]8;id=962030;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=365002;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#382\382]8;;\

...

                    INFO     Loading data from 'y_pred' (MemoryDataSet)...                      ]8;id=341188;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=715430;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'y_test' (MemoryDataSet)...                      ]8;id=167221;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=474055;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Running node: report_accuracy: report_accuracy([y_pred,y_test]) -> None    ]8;id=763729;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py\node.py]8;;\:]8;id=630318;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py#327\327]8;;\

                    INFO     Model has accuracy of 0.933 on test data.                                  ]8;id=591339;file:///Users/Nok_Lam_Chan/dev/kedro_gallery/kedro-debug-runner-demo/src/kedro_debug_runner_demo/nodes.py\nodes.py]8;;\:]8;id=197775;file:///Users/Nok_Lam_Chan/dev/kedro_gallery/kedro-debug-runner-demo/src/kedro_debug_runner_demo/nodes.py#74\74]8;;\

                    INFO     Loading data from 'example_iris_data' (CSVDataSet)...              ]8;id=83191;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=52801;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

In [ ]:
runner = DebugRunner()
default_pipeline = pipelines["__default__"]
run_3 = runner.run(default_pipeline, catalog, dataset_names=["X_train"]) # Input datasets


                    INFO     Loading data from 'example_iris_data' (CSVDataSet)...              ]8;id=402027;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=61133;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'parameters' (MemoryDataSet)...                  ]8;id=544469;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=15026;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Running node: split: split_data([example_iris_data,parameters]) ->         ]8;id=762351;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py\node.py]8;;\:]8;id=453646;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py#327\327]8;;\
                             [X_train,X_test,y_train,y_test]                                                       

                    INFO     Saving data to 'X_train' (MemoryDataSet)...                        ]8;id=928771;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=857479;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#382\382]8;;\

                    INFO     Saving data to 'X_test' (MemoryDataSet)...                         ]8;id=516693;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=591157;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#382\382]8;;\

...

                    INFO     Loading data from 'y_pred' (MemoryDataSet)...                      ]8;id=638118;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=538458;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'y_test' (MemoryDataSet)...                      ]8;id=338929;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=462314;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Running node: report_accuracy: report_accuracy([y_pred,y_test]) -> None    ]8;id=782343;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py\node.py]8;;\:]8;id=772106;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/pipeline/node.py#327\327]8;;\

                    INFO     Model has accuracy of 0.933 on test data.                                  ]8;id=59537;file:///Users/Nok_Lam_Chan/dev/kedro_gallery/kedro-debug-runner-demo/src/kedro_debug_runner_demo/nodes.py\nodes.py]8;;\:]8;id=61229;file:///Users/Nok_Lam_Chan/dev/kedro_gallery/kedro-debug-runner-demo/src/kedro_debug_runner_demo/nodes.py#74\74]8;;\

                    INFO     Loading data from 'X_train' (MemoryDataSet)...                     ]8;id=33632;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=756401;file:///Users/Nok_Lam_Chan/GitHub/kedro/kedro/io/data_catalog.py#343\343]8;;\

In [ ]:
run_1

{}

In [ ]:
run_2

{
    'example_iris_data':      sepal_length  sepal_width  petal_length  petal_width    species
0             5.1          3.5           1.4          0.2     setosa
1             4.9          3.0           1.4          0.2     setosa
2             4.7          3.2           1.3          0.2     setosa
3             4.6          3.1           1.5          0.2     setosa
4             5.0          3.6           1.4          0.2     setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  virginica
146           6.3          2.5           5.0          1.9  virginica
147           6.5          3.0           5.2          2.0  virginica
148           6.2          3.4           5.4          2.3  virginica
149           5.9          3.0           5.1          1.8  virginica

[150 rows x 5 columns]
}

In [ ]:
run_3

{
    'X_train':      sepal_length  sepal_width  petal_length  petal_width
47            4.6          3.2           1.4          0.2
3             4.6          3.1           1.5          0.2
31            5.4          3.4           1.5          0.4
25            5.0          3.0           1.6          0.2
15            5.7          4.4           1.5          0.4
..            ...          ...           ...          ...
28            5.2          3.4           1.4          0.2
78            6.0          2.9           4.5          1.5
146           6.3          2.5           5.0          1.9
49            5.0          3.3           1.4          0.2
94            5.6          2.7           4.2          1.3

[120 rows x 4 columns]
}

In [ ]:
#| export


class GreedySequentialRunner(SequentialRunner):
    def run(
        self,
        pipeline: Pipeline,
        catalog: DataCatalog,
        hook_manager: PluginManager = None,
        session_id: str = None,
    ) -> Dict[str, Any]:
        """Run the ``Pipeline`` using the datasets provided by ``catalog``
        and save results back to the same objects.

        Args:
            pipeline: The ``Pipeline`` to run.
            catalog: The ``DataCatalog`` from which to fetch data.
            hook_manager: The ``PluginManager`` to activate hooks.
            session_id: The id of the session.

        Raises:
            ValueError: Raised when ``Pipeline`` inputs cannot be satisfied.

        Returns:
            Any node outputs that cannot be processed by the ``DataCatalog``.
            These are returned in a dictionary, where the keys are defined
            by the node outputs.

        """

        hook_manager = hook_manager or _NullPluginManager()
        catalog = catalog.shallow_copy()

        unsatisfied = pipeline.inputs() - set(catalog.list())
        if unsatisfied:
            raise ValueError(
                f"Pipeline input(s) {unsatisfied} not found in the DataCatalog"
            )

        free_outputs = pipeline.outputs() # Return everything regardless if it it's in catalog
        unregistered_ds = pipeline.data_sets() - set(catalog.list())
        for ds_name in unregistered_ds:
            catalog.add(ds_name, self.create_default_data_set(ds_name))

        if self._is_async:
            self._logger.info(
                "Asynchronous mode is enabled for loading and saving data"
            )
        self._run(pipeline, catalog, hook_manager, session_id)

        self._logger.info("Pipeline execution completed successfully.")

        return {ds_name: catalog.load(ds_name) for ds_name in free_outputs}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()